In [22]:
import torch
import numpy as np
import mindspore
from mindspore import Tensor, dtype
from mindspore import save_checkpoint
from mindspore import Parameter

name = "resnet101_torch"

model = torch.load(f"{name}.pth", map_location="cpu")

In [23]:
num = 0
params_num = 0
for key in model:
    if "num_batches_tracked" in key or key in ["fc.bias", "fc.weight"]:
        continue
    num += 1
    params_num += model[key].numel()
    print(key, model[key].shape)
print(num, params_num)

conv1.weight torch.Size([64, 3, 7, 7])
bn1.running_mean torch.Size([64])
bn1.running_var torch.Size([64])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 1, 1])
layer1.0.bn1.running_mean torch.Size([64])
layer1.0.bn1.running_var torch.Size([64])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.running_mean torch.Size([64])
layer1.0.bn2.running_var torch.Size([64])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.0.conv3.weight torch.Size([256, 64, 1, 1])
layer1.0.bn3.running_mean torch.Size([256])
layer1.0.bn3.running_var torch.Size([256])
layer1.0.bn3.weight torch.Size([256])
layer1.0.bn3.bias torch.Size([256])
layer1.0.downsample.0.weight torch.Size([256, 64, 1, 1])
layer1.0.downsample.1.running_mean torch.Size([256])
layer1.0.downsample.1.running_var torch.Size([256])
layer1.0.downsample.1.weight torch.Size([256])
layer1.0

In [24]:
weights = []
prefix = "model."
k = 0
for key in model:
    if "num_batches_tracked" in key or key in ["fc.bias", "fc.weight"]:
        continue
    name2weight = {}
    if "bn" in key or "norm" in key or "ln" in key:
        if "weight" in key:
            name2weight["name"] = prefix + key.replace(".weight", ".gamma")
        elif "bias" in key:
            name2weight["name"] = prefix + key.replace(".bias", ".beta")
        elif "mean" in key:
            name2weight["name"] = prefix + key.replace("running_mean", "moving_mean")
        elif "var" in key:
            name2weight["name"] = prefix + key.replace("running_var", "moving_variance")
        
        name2weight["data"] = Parameter(Tensor(model[key].detach().numpy(), dtype.float32),requires_grad=True) 
        weights.append(name2weight)
    else:
        data_type = dtype.float32
        weight = Parameter(Tensor(model[key].detach().numpy(), data_type),requires_grad=True)
        temp = prefix + key
        weights.append({"name": temp, "data": weight})
save_checkpoint(weights, f"{name}.ckpt")